In [11]:
using Distributions

function meanvarstdskku(dist)
    m = mean(dist)
    s2 = var(dist)
    s = √s2
    sk = skewness(dist)
    ku = kurtosis(dist)
    m, s2, s, sk, ku
end

function meanvarstdskku(dist::MixtureModel)
    M1, M2, M3, M4 = moment1234(dist)
    m = mean(dist)
    s2 = var(dist)
    s = √s2
    s3, s4 = s*s2, s2^2
    sk = 1/s3 * (M3 - 3m*s2 - m^3)
    ku = 1/s4 * (M4 - 4m*s3*sk - 6m^2*s2 - m^4) - 3
    m, s2, s, sk, ku
end

function moment1234(dist)
    m, s2, s, sk, ku = meanvarstdskku(dist)
    s3, s4 = s*s2, s2^2
    m1 = m
    m2 = s2 + m^2
    m3 = s3*sk + 3m*s2 + m^3
    m4 = s4*(ku + 3) + 4m*s3*sk + 6m^2*s2 + m^4
    m1, m2, m3, m4
end

function moment1234(dist::MixtureModel)
    T = float(eltype(dist))
    M1, M2, M3, M4 = zero(T), zero(T), zero(T), zero(T)
    for (d, q) in zip(components(dist), probs(dist))
        m1, m2, m3, m4 = moment1234(d)
        M1 += q * m1
        M2 += q * m2
        M3 += q * m3
        M4 += q * m4
    end
    M1, M2, M3, M4
end

# Warning: type piracy
Distributions.skewness(dist::MixtureModel) = meanvarstdskku(dist)[4]
Distributions.kurtosis(dist::MixtureModel) = meanvarstdskku(dist)[5]

In [2]:
mix = MixtureModel([Normal(), Normal(20)], [0.95, 0.05])
@show skewness(mix) kurtosis(mix);

skewness(mix) = 3.8236762415246406
kurtosis(mix) = 13.585


In [3]:
mixdisc = MixtureModel([Poisson(), Poisson(10)], [0.9, 0.1])
@show skewness(mixdisc) kurtosis(mixdisc);

skewness(mixdisc) = 2.737235374074743
kurtosis(mixdisc) = 7.571784631305494


In [4]:
mixmix = MixtureModel([mix, Normal(2, 3)], [0.7, 0.3])
@show skewness(mixmix) kurtosis(mixmix);

skewness(mixmix) = 3.3443244390341396
kurtosis(mixmix) = 12.544763892468605


In [5]:
using Distributions
using QuadGK

function skewness_approx(dist::ContinuousUnivariateDistribution)
    μ = mean(dist)
    σ = std(dist)
    quadgk(x -> ((x-μ)/σ)^3 * pdf(dist, x), extrema(dist)...)[1]
end

function skewness_approx(dist::DiscreteUnivariateDistribution)
    μ = mean(dist)
    σ = std(dist)
    xmin = max(round(Int, μ-100σ), minimum(dist))
    xmax = min(round(Int, μ+100σ), maximum(dist))
    sum(x -> ((x-μ)/σ)^3 * pdf(dist, x), xmin:xmax)
end

function kurtosis_approx(dist::ContinuousUnivariateDistribution)
    μ = mean(dist)
    σ = std(dist)
    quadgk(x -> ((x-μ)/σ)^4 * pdf(dist, x), extrema(dist)...)[1] - 3
end

function kurtosis_approx(dist::DiscreteUnivariateDistribution)
    μ = mean(dist)
    σ = std(dist)
    xmin = max(round(Int, μ-100σ), minimum(dist))
    xmax = min(round(Int, μ+100σ), maximum(dist))
    sum(x -> ((x-μ)/σ)^4 * pdf(dist, x), xmin:xmax) - 3
end

function meanvarstdskku_approx(dist)
    m = mean(dist)
    s2 = var(dist)
    s = √s2
    sk = skewness_approx(dist)
    ku = kurtosis_approx(dist)
    m, s2, s, sk, ku
end

function _moment_approx(dist::ContinuousUnivariateDistribution, k)
    quadgk(x -> x^k * pdf(dist, x), extrema(dist)...)[1]
end

function _moment_approx(dist::DiscreteUnivariateDistribution, k)
    μ = mean(dist)
    σ = std(dist)
    xmin = max(round(Int, μ-100σ), minimum(dist))
    xmax = min(round(Int, μ+100σ), maximum(dist))
    sum(x -> x^k * pdf(dist, x), xmin:xmax)
end

function moment1234_approx(dist)
    m1, m2, m3, m4 = (_moment_approx(dist, k) for k in 1:4)
    m1, m2, m3, m4
end

moment1234_approx (generic function with 1 method)

In [6]:
dist = Gamma(2, 3)
@show meanvarstdskku(dist) meanvarstdskku_approx(dist)
@show moment1234(dist) moment1234_approx(dist);

meanvarstdskku(dist) = (6.0, 18.0, 4.242640687119285, 1.414213562373095, 3.0)
meanvarstdskku_approx(dist) = (6.0, 18.0, 4.242640687119285, 1.4142135623731023, 2.9999999999999982)
moment1234(dist) = (6.0, 54.0, 648.0, 9720.0)
moment1234_approx(dist) = (5.999999999999999, 54.000000000000014, 648.0000000000008, 9719.999999999909)


In [7]:
dist = Poisson(30)
@show meanvarstdskku(dist) meanvarstdskku_approx(dist)
@show moment1234(dist) moment1234_approx(dist);

meanvarstdskku(dist) = (30.0, 30.0, 5.477225575051661, 0.18257418583505536, 0.03333333333333333)
meanvarstdskku_approx(dist) = (30.0, 30.0, 5.477225575051661, 0.1825741858350566, 0.03333333333334343)
moment1234(dist) = (30.0, 930.0, 29730.0, 978330.0)
moment1234_approx(dist) = (30.000000000000053, 930.000000000002, 29730.000000000073, 978330.000000002)


In [8]:
mix = MixtureModel([Normal(), Normal(20)], [0.95, 0.05])
@show moment1234(mix)
@show moment1234_approx(mix)
@show meanvarstdskku(mix)
@show meanvarstdskku_approx(mix);

moment1234(mix) = (1.0, 21.0, 403.0, 8123.000000000001)
moment1234_approx(mix) = (0.9999999999999988, 20.999999999999982, 403.00000000022675, 8122.999999997206)
meanvarstdskku(mix) = (1.0, 20.0, 4.47213595499958, 3.8236762415246406, 13.585)
meanvarstdskku_approx(mix) = (1.0, 20.0, 4.47213595499958, 3.823676241524649, 13.584999999990973)


In [9]:
mixdisc = MixtureModel([Poisson(1), Poisson(10)], [0.9, 0.1])
@show moment1234(mixdisc)
@show moment1234_approx(mixdisc)
@show meanvarstdskku(mixdisc)
@show meanvarstdskku_approx(mixdisc);

moment1234(mixdisc) = (1.9, 12.8, 135.5, 1684.5)
moment1234_approx(mixdisc) = (1.9000000000000035, 12.800000000000038, 135.50000000000045, 1684.5000000000068)
meanvarstdskku(mixdisc) = (1.9, 9.19, 3.0315012782448236, 2.737235374074743, 7.571784631305494)
meanvarstdskku_approx(mixdisc) = (1.9, 9.19, 3.0315012782448236, 2.737235374074752, 7.5717846313055315)


In [10]:
mixmix = MixtureModel([mix, Normal(2, 3)], [0.7, 0.3])
@show moment1234(mixmix)
@show moment1234_approx(mixmix)
@show meanvarstdskku(mixmix)
@show meanvarstdskku_approx(mixmix);

moment1234(mixmix) = (1.2999999999999998, 18.599999999999998, 300.7, 5828.6)
moment1234_approx(mixmix) = (1.299999999999999, 18.599999999999987, 300.7000000000006, 5828.600000000012)
meanvarstdskku(mixmix) = (1.2999999999999998, 16.909999999999997, 4.112177038990417, 3.3443244390341396, 12.544763892468605)
meanvarstdskku_approx(mixmix) = (1.2999999999999998, 16.909999999999997, 4.112177038990417, 3.344324439034147, 12.544763892468545)
